<a href="https://colab.research.google.com/github/fatemafaria142/Leveraging-Large-Language-Models-for-Comprehensive-Math-Word-Problem-Solving/blob/main/Math_Word_Problem_Solving_using_Mistral_7B_Instruct_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00


#### **Dataset Link:**   https://huggingface.co/datasets/camel-ai/math

In [ ]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("camel-ai/math")

# Filter the dataset to include only instances where the topic is "Algebra" or "Trigonometry"
instruct_tune_dataset['train'] = instruct_tune_dataset['train'].filter(lambda example: example['topic;'] in ['Algebra', 'Trigonometry'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

### **Dataset structure**
* The dataset contains five columns.

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 4000
    })
})

In [ ]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Role:", data['role_1'])
    print("Topic:", data['topic;'])
    print("Sub Topic:", data['sub_topic'])
    print("Message 1:", data['message_1'])
    print("Message 2:", data['message_2'])
    print("\n-----------------------------\n")

Data Point 1:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 5x + 3 = 28
Message 2: To solve for x, we need to isolate x on one side of the equation. 

Step 1: Subtract 3 from both sides of the equation.
5x + 3 - 3 = 28 - 3
5x = 25

Step 2: Divide both sides of the equation by 5.
5x / 5 = 25 / 5
x = 5

So, x = 5.

-----------------------------

Data Point 2:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 2x + 5 = 9- x.
Message 2: To solve for x, we will follow these steps:

1. Add x to both sides of the equation to get all the x terms on one side:
2x + x + 5 = 9 - x + x

This simplifies to:
3x + 5 = 9

2. Subtract 5 from both sides of the equation to isolate the x terms:
3x + 5 - 5 = 9 - 5

This simplifies to:
3x = 4

3. Divide both sides of the equation by 3 to solve for x:
3x / 3 = 4 / 3

This simplifies to:
x = 4/3

So, the solution is x = 4/3.

-----

### **We will use just a small subset of the data for this training example**

In [ ]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(4000))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(400))

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 400
    })
})

* Note that this time, the tokenizer has added the control tokens `[INST]` and `[/INST]` to indicate the start and end of user messages (but not assistant messages!). **Mistral-instruct was trained with these tokens.**
* In order to leverage instruction fine-tuning, your prompt should be surrounded by `[INST]` and `[/INST]` tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

#### **Prompt Creation**

In [ ]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    eos_token = "</s>"

    # Use a predefined template for instructions
    instructions_template = "Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "[INST]"
    full_prompt += instructions_template
    full_prompt += "My Question is: "
    full_prompt += sample['message_1']
    full_prompt += "[/INST]"
    full_prompt += sample['message_2']
    full_prompt += eos_token

    return full_prompt

In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 5x + 3 = 28[/INST]To solve for x, we need to isolate x on one side of the equation. \n\nStep 1: Subtract 3 from both sides of the equation.\n5x + 3 - 3 = 28 - 3\n5x = 25\n\nStep 2: Divide both sides of the equation by 5.\n5x / 5 = 25 / 5\nx = 5\n\nSo, x = 5.</s>'

In [ ]:
create_prompt(instruct_tune_dataset["train"][1])

'<s>[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 2x + 5 = 9- x.[/INST]To solve for x, we will follow these steps:\n\n1. Add x to both sides of the equation to get all the x terms on one side:\n2x + x + 5 = 9 - x + x\n\nThis simplifies to:\n3x + 5 = 9\n\n2. Subtract 5 from both sides of the equation to isolate the x terms:\n3x + 5 - 5 = 9 - 5\n\nThis simplifies to:\n3x = 4\n\n3. Divide both sides of the equation by 3 to solve for x:\n3x / 3 = 4 / 3\n\nThis simplifies to:\nx = 4/3\n\nSo, the solution is x = 4/3.</s>'

In [ ]:
create_prompt(instruct_tune_dataset["train"][2])

'<s>[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation: 3x + 5 = 14.[/INST]To solve the equation 3x + 5 = 14, we need to isolate the variable x. \n\nStep 1: Subtract 5 from both sides of the equation.\n3x + 5 - 5 = 14 - 5\n3x = 9\n\nStep 2: Divide both sides of the equation by 3.\n3x / 3 = 9 / 3\nx = 3\n\nSo, the solution to the equation 3x + 5 = 14 is x = 3.</s>'

In [ ]:
create_prompt(instruct_tune_dataset["train"][200])

"<s>[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the following system of linear equations using substitution:\n\n2x + 3y = 7 \n\n5x - 4y = -23[/INST]First, we will solve one of the equations for one of the variables. Let's solve the first equation for x:\n\n2x + 3y = 7\n\n2x = 7 - 3y\n\nx = (7 - 3y) / 2\n\nNow, we will substitute this expression for x into the second equation:\n\n5x - 4y = -23\n\n5((7 - 3y) / 2) - 4y = -23\n\nNow, we will solve for y:\n\n(35 - 15y) / 2 - 4y = -23\n\nMultiply both sides by 2 to get rid of the fraction:\n\n35 - 15y - 8y = -46\n\nCombine like terms:\n\n-23y = -81\n\nNow, divide by -23:\n\ny = 81 / 23\n\ny = 3\n\nNow that we have the value for y, we can substitute it back into the expression we found for x:\n\nx = (7 - 3y) / 2\n\n

In [ ]:
create_prompt(instruct_tune_dataset["train"][2])

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer Link:** https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [ ]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, top_k=5,temperature=0.7, repetition_penalty=1.2,do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
# Use a predefined template for instructions
prompt = "[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: "
prompt += "Solve the following system of linear equations using substitution:\n\n2x + 3y = 7 \n\n5x - 4y = -23 [/INST]" #message_1
print(prompt)

[INST]Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the following system of linear equations using substitution:

2x + 3y = 7 

5x - 4y = -23 [/INST]


In [ ]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s>  To solve this system of linear equations using substitution, we need to isolate one variable in one equation and substitute it into the other equation. Let's start by solving for x in the first equation:\n\n1. Multiply both sides of the first equation by -3 to eliminate y:\n\n-3(2x + 3y) = -3(7)\n(-6x - 9y) = -21\n\n2. Add 9y to both sides of the new equation:\n\n-6x + 9y = -21 + 9y\n-6x + 0y = -12\n\nSo, we have x = 2. Now that we know x = 2, we can substitute x into either equation to find y. We will use the second equation:\n\n1. Substitute x = 2 into the second equation:\n\n5(2) - 4y = -23\n10 - 4y = -23\n\n2. Solve for y:\n\n-4y = -23 - 10\n-4y = -33\n\nDivide by -4:\n"

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!
* `r (int):` Lora attention dimension.
* `lora_alpha (int):` The alpha parameter for Lora scaling.
* `lora_dropout (float):` The dropout probability for Lora layers.
* `bias (str):` Bias type for Lora. Can be 'none', 'all' or 'lora_only'. If 'all' or 'lora_only', the

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
#This is the configuration class to store the configuration of a [LoraModel].
peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [ ]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [ ]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.322600
20,0.893800
30,0.689600
40,0.614100
50,0.576600
60,0.646700
70,0.617600
80,0.569900
90,0.581900
100,0.581700


TrainOutput(global_step=150, training_loss=0.6544661617279053, metrics={'train_runtime': 748.6706, 'train_samples_per_second': 0.801, 'train_steps_per_second': 0.2, 'total_flos': 6556325039308800.0, 'train_loss': 0.6544661617279053, 'epoch': 0.1})

### **Save the model**

In [29]:
trainer.save_model("mistral_instruct_generation")

In [30]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


### **Example No:1**

In [31]:
# Use a predefined template for instructions
prompt = "[INST] Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My question is:"
prompt +=" Solve the following rational equation for x: \n\n(2x-1)/(x+3) = 5/2 [/INST]"
response = generate_response(prompt, merged_model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>  To solve this problem, we will use algebraic manipulation and substitution methods.

Step 1: Multiply both sides of the equation by (x + 3) to eliminate the denominator:

Multiplying both sides by (x + 3)
----------------------------------
(2x - 1)(x + 3) = 5(x + 3)
Simplifying the left side using FOIL method
---------------------------------------------
2x^2 + 6x - 1 = 5x + 15
Subtracting 5x from both sides
------------------------------
2x^2 + 6x - 5x - 1 = 5x + 15 - 5x
Simplifying further
---------------------
x^2 + 1 = 15

Now that we have the quadratic equation in standard form, we can proceed with solving for x.

Step 2: Factor the quadratic equation:

Factoring the equation x² + 1 = 15
--------------------------------------
(x + √


### **Example No:2**

In [32]:
# Use a predefined template for instructions
prompt = "[INST] Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My question is:"
prompt += " Solve the inequality 2x + 5 > 11, and express the solution set in interval notation. [/INST]"
response = generate_response(prompt, merged_model)
# Print the response with formatted output
print(response)

<s>  To solve the inequality $2x+5>11$, we can subtract $5$ from both sides of the inequality:
$$2x+5-5>11-5.$$
This simplifies to:
$$2x>6.$$
Now, we can divide both sides by $2$:
$$\frac{2x}{2}>\frac{6}{2}.$$
This simplifies to:
$$x>\frac{3}{2}.$$
Therefore, the solution set is $\boxed{\left(\frac{3}{2},\infty\right)}$.</s>


### **Example No:3**

In [33]:
# Use a predefined template for instructions
prompt = "[INST] Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My question is:"
prompt +=" Factor the polynomial expression: x^2 + 7x + 10. [/INST]"
response = generate_response(prompt, merged_model)
# Print the response with formatted output
print(response)

<s>  To factor the polynomial expression $x^2+7x+10$, we need to find two numbers whose product equals 10 and whose sum equals -7. These numbers are -5 and -2.

Now, let's use these numbers to rewrite the polynomial expression in terms of binomials:

$x^2+7x+10=x^2-5x-2(7x+5)$

Next, we will distribute the negative sign inside the second parentheses:

$x^2+7x+10=(x-5)(7x+5)$

So, the factored form of the polynomial expression $x^2+7x+10$ is $(x-5)(7x+5)$.</s>
